In [10]:
import pandas as pd
import os

In [12]:
import pandas as pd
import os

# Cargar la base de datos
file_path = "Base_Datos_Unificada.xlsx"
df = pd.read_excel(file_path, dtype=str)

# **Eliminar el sufijo '- 0' en los nombres antes de cualquier otro paso**
df["INSTITUCIÓN"] = df["INSTITUCIÓN"].str.replace(r" - \d+$", "", regex=True).str.strip()

# **Eliminar filas donde Departamento, Provincia, Distrito o Domicilio estén vacíos**
df = df.dropna(subset=["DEPARTAMENTO", "PROVINCIA", "DISTRITO", "DOMICILIO"])

# **Crear la tabla 'Instituciones'**
instituciones = df[["INSTITUCIÓN", "SIGLAS"]].drop_duplicates().reset_index(drop=True)
instituciones.insert(0, "id_institucion", instituciones.index + 1)  # Clave primaria al inicio

# **Crear la tabla 'Distrito' (Ubicación Administrativa), ordenando correctamente**
distritos = df[["DISTRITO", "PROVINCIA", "DEPARTAMENTO"]].drop_duplicates()
distritos = distritos.sort_values(by=["DEPARTAMENTO", "PROVINCIA", "DISTRITO"]).reset_index(drop=True)
distritos.insert(0, "id_distrito", distritos.index + 1)  # Clave primaria al inicio

# **Crear la tabla 'Dirección' (Dirección específica)**
direcciones = df[["INSTITUCIÓN", "DOMICILIO", "DISTRITO"]].drop_duplicates()
direcciones = direcciones.merge(instituciones, on="INSTITUCIÓN", how="left")  # Unir con instituciones
direcciones = direcciones.merge(distritos, on="DISTRITO", how="left")  # Obtener id_distrito
direcciones = direcciones[["DOMICILIO", "id_distrito", "id_institucion"]].drop_duplicates().reset_index(drop=True)
direcciones.insert(0, "id_direccion", direcciones.index + 1)  # Clave primaria al inicio

# **Crear la tabla 'Áreas'**
df["ÁREA"] = df["ÁREA"].fillna("")  # Evitar valores NaN
df["ÁREA"] = df["ÁREA"].str.split(" / ")  # Dividir los valores múltiples en listas
areas_exploded = df.explode("ÁREA")[["ÁREA"]].drop_duplicates().reset_index(drop=True)
areas_exploded.insert(0, "id_area", areas_exploded.index + 1)  # Clave primaria al inicio

# **Crear la tabla intermedia 'Institucion_Area'**
institucion_area = df.explode("ÁREA")[["INSTITUCIÓN", "ÁREA"]].drop_duplicates()
institucion_area = institucion_area.merge(instituciones, on="INSTITUCIÓN", how="left")
institucion_area = institucion_area.merge(areas_exploded, on="ÁREA", how="left")
institucion_area = institucion_area[["id_institucion", "id_area"]].drop_duplicates().reset_index(drop=True)
institucion_area.insert(0, "id_institucion_area", institucion_area.index + 1)  # Clave primaria al inicio

# **Crear la tabla 'Semestres'**
semestres = df[["INSTITUCIÓN", "PRIMER SEMESTRE", "ÚLTIMO SEMESTRE"]].drop_duplicates()
semestres = semestres.merge(instituciones, on="INSTITUCIÓN", how="left")
semestres = semestres[["id_institucion", "PRIMER SEMESTRE", "ÚLTIMO SEMESTRE"]].drop_duplicates().reset_index(drop=True)
semestres.insert(0, "id_semestre", semestres.index + 1)  # Clave primaria al inicio

# **Ruta de salida**
output_folder = "180 Postulación/Normalizada"
os.makedirs(output_folder, exist_ok=True)

# **Ruta del archivo Excel combinado**
output_file = f"{output_folder}/Base_Datos_Normalizada.xlsx"

# **Guardar todas las tablas en un solo archivo Excel con múltiples hojas**
with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    instituciones.to_excel(writer, sheet_name="Instituciones", index=False)
    distritos.to_excel(writer, sheet_name="Distrito", index=False)
    direcciones.to_excel(writer, sheet_name="Direcciones", index=False)
    areas_exploded.to_excel(writer, sheet_name="Areas", index=False)
    institucion_area.to_excel(writer, sheet_name="Institucion_Area", index=False)
    semestres.to_excel(writer, sheet_name="Semestres", index=False)

print("Base de datos normalizada y guardada en un solo archivo Excel con múltiples hojas correctamente")


Base de datos normalizada y guardada en un solo archivo Excel con múltiples hojas correctamente
